# SQLITE test

With this notebook, we set up a multi-thread data transfer to obtain several years of climate data from the NOAA GCHD repository. A sqlite database is set up to track the progress of data transfers and record success or failure. 

Once you were successful, you need to manually delete the database file downloads.db to execute again.

Hints for debugging:
1. to avoid unnecessary network traffic, set the debug=True option in manage_download.
2. Multithreading comes with the price that debugging is somewhat more complicated, because (without special configuration) any failure in execution will be silently ignored.

In [ ]:
import sqlite3
import requests
from concurrent.futures import ThreadPoolExecutor
from functools import partial

In [ ]:
# user settings
yearrange = [1960, 1990]

In [ ]:
# setup database to keep track of downloads
con = sqlite3.connect("downloads.db")
cur = con.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS downloads(year INTEGER PRIMARY KEY, status TEXT, tries INTEGER)")
for y in range(yearrange[0], yearrange[1]):
    try:
        cur.execute(f"""INSERT INTO downloads VALUES
            ({y}, 'unknown', 0)""")
    except:
        cmd = f"UPDATE downloads SET status='unknown' WHERE year={y}"
        cur.execute(cmd)
    con.commit()
        
# check DB content
res = cur.execute("SELECT * FROM downloads")
print(res.fetchall())

# DB methods
def get_status(year, cur=cur):
    print("get_status", cur)
    res = cur.execute(f"SELECT status FROM downloads WHERE year={year}")
    print(res)
    status = res.fetchone()
    return status[0]
    
def update_db(year, status, cur=cur):
    cmd = f"UPDATE downloads SET status='{status}' WHERE year={year}"
    print(cmd)
    cur.execute(cmd)
    con = cur.connection
    con.commit()

def increase_tries(year, cur=cur):
    res = cur.execute(f"SELECT tries FROM downloads WHERE year={year}")
    tries = res.fetchone()[0]
    cmd = f"UPDATE downloads SET tries={tries+1} WHERE year={year}"
    print(cmd)
    cur.execute(cmd)
    con = cur.connection
    con.commit()
    

In [ ]:
# data download from NOAA
def download(year):
    url = f"https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/{year}.csv.gz"
    print(url)
    response = requests.get(url)
    if response.ok:
        filename = url.rsplit('/', 1)[1]
        print(f"data downloaded. Will be saved as {filename}")
        with open(filename, "wb") as f:
            f.write(response.content)
        return 'done'
    else:
        print(f"An error occured while trying to retrieve the data of year {year} from the internet.")
        return 'failed'


def fake_download(year):
    # for debugging. Don't actually load data, but simulate transfer failures occasionally.
    url = f"***https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/{year}.csv.gz"
    print(url)
    if (year % 5 == 0):
        return "failed"
    else:
        return "done"

def manage_download(year, dbname="downloads.db"):
    debug=False
    # need to open a new connection each time to have this thread-safe
    con = sqlite3.connect(dbname)
    cur = con.cursor()
    # check if year was already transferred
    status = get_status(year, cur=cur)
    print(f"processing year {year}: status={status}")
    if status == 'done':
        print(f"Skipping year {year}, because it was already transferred.")
        return
    update_db(year, "in progress", cur=cur)
    increase_tries(year, cur=cur)
    status = fake_download(year) if debug else download(year)
    update_db(year, status, cur=cur)


In [ ]:
# main loop
with ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(manage_download, range(yearrange[0], yearrange[1]))

In [ ]:
# for testing: uncomment and execute if needed
# manage_download(1966)

In [ ]:
res = cur.execute("SELECT * FROM downloads")
for line in res.fetchall():
    print(line)

In [ ]:
# for testing: uncomment and execute if needed
# get_status(1971, cur=cur)